In [5]:
import import_ipynb
import numpy as np
import Chapter5_backpropagation

In [10]:
#Relu activate function
# y=x (x>0), y=0 (x <= 0)
# dy/dx = 1 (x>0), dy/dx = 0 (x <= 0)

class Relu(BaseLayer):
    def __init__(self):
        self.mask = None
    
    def forward(self, x, y=None):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

In [17]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
relu_layer = Relu()
out = relu_layer.forward(x)
print(relu_layer.mask)
print('out: ', out)

[[False  True]
 [ True False]]
[[False  True]
 [ True False]]
out:  [[1. 0.]
 [0. 3.]]


In [24]:
print(relu_layer.backward(np.array([[1, 1], [1, 1]])))

[[1 0]
 [0 1]]


In [25]:
#sigmoid activate function
# y = 1/(1+exp(-x))
class Sigmoid(BaseLayer):
    def __init__(self):
        self.out = None
    
    def forward(self, x, y=None):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out #sigmoid 의 미분 값은 출력값만으로 계산할 수 있음.
        return dx

In [26]:
#Affine and Softmax
#Affine은 WX + b 를 수행하는 연산을 의미함.

class Affine(BaseLayer):
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x, y=None):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        return dx


In [70]:
#출력 정규화를 위한 softmax with loss 함수 구현
class Softmax(BaseLayer):
    def __init__(self):
        self.args = None
        self.S = None
        self.dS = None
        self.dargs = None
        self.out = None
        self.dout = None
    
    def forward(self, x, y=None):
        self.args = x
        exp_val = np.exp(self.args)
        self.S = np.sum(exp_val)
        self.out = exp_val/self.S
        return self.out
    
    def backward(self, dout):
        # -t/y 가 흘러들어옴
        self.dout = self.out * (1+dout)
        return self.dout
        

In [71]:
print(Softmax().forward(np.array([0.3, 2.9, 4.0])))

[0.01821127 0.24519181 0.73659691]


In [72]:
class CrossEntropyErrorLayer(BaseLayer):
    def __init__(self):
        self.y = None
        self.t = None
        self.dot_val = None
        self.sum_val = None
        self.mul_minusone = None
        
    def forward(self, y, t):
        self.y = y
        self.t = t
        delta = 1e-7
        self.dot_val = np.dot(np.log(self.y + delta), t)
        self.sum_val = np.sum(self.dot_val)
        self.mul_minusone = self.sum_val * (-1)
        return self.mul_minusone
    
    def backward(self, dout):
        
        self.dout = -1 * self.t / (self.y+1e-7)
        return self.dout
        

In [73]:
cee = CrossEntropyErrorLayer()
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
res = cee.forward(np.array(y), np.array(t))
print(cee.backward(1))

[ 0  0 -1  0  0  0  0  0  0  0]
[ 0.          0.         -1.66666639  0.          0.          0.
  0.          0.          0.          0.        ]


In [77]:
arr = [0.3, 2.9, 4.0]
sm = Softmax()
cee = CrossEntropyErrorLayer()
L = cee.forward(sm.forward(np.array(arr)), np.array([0, 0, 1]))
print(L)

0.30571432905300044


In [78]:
dL = sm.backward(cee.backward(1))
print(dL)

[ 0  0 -1]
[ 0.01821127  0.24519181 -0.26340295]
